In [ ]:
from tensorflow.keras.layers import StringLookup
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
import pytesseract
import tkinter as tk
from PIL import Image, ImageDraw, ImageTk, ImageEnhance, ImageFilter
from tkinter import messagebox

In [ ]:
model = keras.models.load_model("Data/model/")

In [ ]:
import tkinter as tk
from PIL import Image, ImageDraw

class HandwritingRecognizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Handwriting Recognizer")

        # Rajzterület létrehozása
        self.canvas = tk.Canvas(self.root, bg="white", width=400, height=200)
        self.canvas.pack()

        self.canvas.bind("<Button-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw)
        self.canvas.bind("<ButtonRelease-1>", self.stop_draw)

        # Gomb létrehozása a predikció indításához
        self.predict_button = tk.Button(self.root, text="Predict", command=self.predict)
        self.predict_button.pack()

        # Gomb létrehozása a rajz törléséhez
        self.clear_button = tk.Button(self.root, text="Clear", command=self.clear_canvas)
        self.clear_button.pack()

        # RajzoSlás változók
        self.drawing = False
        self.image = Image.new("L", (400, 200), "white")
        self.draw = ImageDraw.Draw(self.image)

        # Eredmény megjelenítése
        self.result_label = tk.Label(self.root, text="", font=("Helvetica", 16))
        self.result_label.pack()

    def start_draw(self, event):
        self.drawing = True
        self.last_x = event.x
        self.last_y = event.y

    def draw(self, event):
        if self.drawing:
            x, y = event.x, event.y
            self.canvas.create_line(self.last_x, self.last_y, x, y, fill="black", width=5)
            self.draw.line((self.last_x, self.last_y, x, y), fill="black", width=5)
            self.last_x = x
            self.last_y = y

    def stop_draw(self, event):
        self.drawing = False


    def preprocess_image(self, image):
        # Kontraszt-növelés és világosság-állítás
        enhanced_image = ImageEnhance.Contrast(image).enhance(1.0)
    
        # Szürkeárnyalat konverzió
        gray_image = enhanced_image.convert("L")
    
        # Küszöbölés
        threshold = 200
        thresholded_image = gray_image.point(lambda p: p > threshold and 255)
    
        # Éldetektálás (Canny éldetektor használata)
        edge_image = thresholded_image.filter(ImageFilter.FIND_EDGES)
    
        return edge_image


    def predict(self):
        # Előfeldolgozási lépések a képen
        preprocessed_image = self.preprocess_image(self.image)
    
        resized_image = preprocessed_image.resize((32, 128), Image.LANCZOS)
        image_array = np.array(resized_image) / 255.0
        image_array = np.expand_dims(image_array, axis=-1)
        image_array = np.expand_dims(image_array, axis=0)
    
        input_label_data = np.array([[1, 2, 3, 4]])  # Példa címke
    
        predictions = model.predict([image_array, input_label_data])
    
        decoded_sequence, _ = keras.backend.ctc_decode(predictions, input_length=np.ones(predictions.shape[0]) * predictions.shape[1])
        output = keras.backend.get_value(decoded_sequence)[0][0]
        predicted_text = ''.join([chr(x) for x in output if x != -1])
    
        # Üzenetablakban jelenítse meg a predikciót
        messagebox.showinfo("Prediction Result", f"Predicted Text: {predicted_text}")


    def clear_canvas(self):
        self.canvas.delete("all")  # Törli a rajzot a vászonról
        self.image = Image.new("L", (128, 512), "white")
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="")  # Törli az eredményt a vászonról

# GUI indítása
root = tk.Tk()
app = HandwritingRecognizerApp(root)
root.mainloop()
